# Transport Usage Prediction

**Team Members**:
- Timur Kasatkin
- Artyom Valeev

**Team name**: Like

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sklearn
import sklearn as skl
import scipy
import pandas as pd
import graphlab
import graphlab as gl
import scikits.crab

ImportError: No module named 'graphlab'

# Data Preprocessing

## Items

In [3]:
items_df = pd.read_csv('items.csv',encoding='cp1251')

/home/timur/dev_ides/anaconda3/envs/gl-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
items_df.dtypes

ItemId            int64
CreatorId         int64
Year             object
DistributorId     int64
dtype: object

In [5]:
items_df[~items_df.Year.str.isnumeric().astype(bool)]

,ItemId,CreatorId,Year,DistributorId
209398,266756,19311,DK Publishing Inc,12172
220500,77452,70208,Gallimard,9591
221447,214498,19311,DK Publishing Inc,638


In [6]:
items_df[items_df.Year.str.isnumeric().astype(bool)].Year.astype(int).describe()

count    270148.000000
mean       1959.756293
std         257.981613
min           0.000000
25%        1989.000000
50%        1995.000000
75%        2000.000000
max        2050.000000
Name: Year, dtype: float64

## Users

In [125]:
users_df = pd.read_csv('users.csv',encoding='cp1251')

In [8]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92106 entries, 0 to 92105
Data columns (total 3 columns):
UserId      92106 non-null int64
Location    92106 non-null object
Age         53887 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [9]:
users_df.Age[users_df.Age.notnull()].describe()

count    53887.000000
mean        35.799711
std         14.809379
min          0.000000
25%         25.000000
50%         33.000000
75%         45.000000
max        244.000000
Name: Age, dtype: float64

## Train

In [10]:
train_prefs_df = pd.read_csv('train_preferences.csv',encoding='cp1251')

train_prefs_df.head()

,UserId,ItemId,Rating
0,25090,129111,0
1,69834,243724,5
2,12098,214334,0
3,27619,250230,3
4,27619,269206,6


In [11]:
'Fraction of no mark provided: %.3f' % (float(train_prefs_df.Rating.value_counts()[0]) / len(train_prefs_df))

'Fraction of no mark provided: 0.660'

In [12]:
print len(users_df.UserId), len(train_prefs_df.UserId.unique())
print len(items_df.ItemId), len(train_prefs_df.ItemId.unique())

92106 87909
270151 262188


## Test

In [13]:
test_prefs_df = pd.read_csv('test_preferences.csv',encoding='cp1251')

test_prefs_df.head()

,PairId,UserId,ItemId
0,0,4345,188337
1,1,58014,164371
2,2,46806,172416
3,3,57654,130724
4,4,5927,63169


# Model evaluation

## Graphlab

In [14]:
def folds(sf,n_splits=5):
    for i in range(0,n_splits):
        yield sf.random_split(1 - float(1)/n_splits)

def cross_val_scores(model_factory,sf,n_splits=10, target='Rating', return_mean=True, verbose=True):
    results = []
    for i, (train_sf, test_sf) in enumerate(folds(sf,n_splits)):
        results.append(model_factory(train_sf).evaluate_rmse(test_sf, target)['rmse_overall'])
        if verbose:
            print 'Iteration %d: RMSE = %f' % (i, results[-1])
    return return_mean and np.mean(results) or results

In [15]:
train_prefs_sf = graphlab.SFrame(train_prefs_df)
train_provided_sf = train_prefs_sf[train_prefs_sf['Rating'] != 0]

This non-commercial license of GraphLab Create for academic use is assigned to t.kasatkin@innopolis.ru and will expire on October 17, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1477226762.log


### Feature selection

In [18]:
cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False, 
                                            num_factors=3,
                                            max_iterations=70,
                                            solver='sgd',
                                            sgd_step_size=0,
                                            side_data_factorization=True,
                                            regularization_type='weighted',
                                            linear_regularization=0.758620689655,
                                            regularization=0.413793103449,
                                            ), train_provided_sf, 5, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.7607451560000988

#### Users

##### Age

In [35]:
exp_user_df = users_df.copy()

exp_user_df.Age.fillna(exp_user_df.Age.median(),inplace=True)

cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(exp_user_df), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 5, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6801152909604262

In [36]:
exp_user_df = users_df.copy()

exp_user_df.Age.fillna(exp_user_df.Age.mean(),inplace=True)

cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(exp_user_df), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 5, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6804037204537667

In [37]:
cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(users_df.copy().drop('Age',axis=1)), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 5, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.758741722849392

In [126]:
users_df.Age = users_df.Age.fillna(users_df.Age.mean())

##### Location

In [50]:
exp_user_df = users_df.copy()

cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(exp_user_df), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 5, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6791922408287774

In [53]:
exp_user_df = users_df.copy()

users_locs_df = exp_user_df.Location.apply(lambda l: pd.Series(l.split(',')), 1)

users_locs_df.rename(columns={0:'City',1:'Province',2:'Country'},inplace=True)

users_locs_df.City = users_locs_df.City.str.strip()
users_locs_df.Province = users_locs_df.Province.str.strip()
users_locs_df.Country = users_locs_df.Country.str.strip()

users_locs_df = users_locs_df[['City','Province','Country']].replace('','n/a')

exp_user_df = pd.concat([exp_user_df,users_locs_df],1)
del exp_user_df['Location']

cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(exp_user_df), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 10, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

...

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.642067073649627

In [54]:
exp_user_df = users_df.copy()

users_locs_df = exp_user_df.Location.apply(lambda l: pd.Series(l.split(',')), 1)

users_locs_df.rename(columns={0:'City',1:'Province',2:'Country'},inplace=True)

users_locs_df.City = users_locs_df.City.str.strip()
users_locs_df.Province = users_locs_df.Province.str.strip()
users_locs_df.Country = users_locs_df.Country.str.strip()

users_locs_df = users_locs_df[['City','Province','Country']].replace('n/a','')

exp_user_df = pd.concat([exp_user_df,users_locs_df],1)
del exp_user_df['Location']

cross_val_scores(lambda train_sf:\
                gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                    user_data=graphlab.SFrame(exp_user_df), 
                                                    num_factors=3,
                                                    max_iterations=70,
                                                    solver='sgd',
                                                    sgd_step_size=0,
                                                    side_data_factorization=True,
                                                    regularization_type='weighted',
                                                    linear_regularization=0.758620689655,
                                                    regularization=0.413793103449,
                                                    ), train_provided_sf, 10, verbose=False)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

...

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6395989204386658

In [127]:
users_locs_df = users_df.Location.apply(lambda l: pd.Series(l.split(',')), 1)

users_locs_df.rename(columns={0:'City',1:'Province',2:'Country'},inplace=True)

users_locs_df.City = users_locs_df.City.str.strip()
users_locs_df.Province = users_locs_df.Province.str.strip()
users_locs_df.Country = users_locs_df.Country.str.strip()

users_locs_df = users_locs_df[['City','Province','Country']].replace('n/a','')

users_df = pd.concat([users_df,users_locs_df],1)

del users_df['Location']

##### Result

**Profit**:
- filling age with median or mean
- splitting location into separate features with stripping all values and replace 'n/a' with empty string

**Bad**:
   - removing Age from users

In [123]:
users_df.head()

,UserId,Age,City,Province,Country
0,4173,18,stockton,california,usa
1,26211,35,timmins,ontario,canada
2,74836,35,germantown,tennessee,usa
3,40801,26,albacete,wisconsin,spain
4,41937,35,fort bragg,california,usa


#### Items

In [68]:
def items_cross_val_scores(item_data, n_splits=5):
    return cross_val_scores(lambda train_sf:\
                            gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                                item_data=item_data,
                                                                num_factors=3,
                                                                max_iterations=70,
                                                                solver='sgd',
                                                                sgd_step_size=0,
                                                                side_data_factorization=True,
                                                                regularization_type='weighted',
                                                                linear_regularization=0.758620689655,
                                                                regularization=0.413793103449,
                                                                ), train_provided_sf, n_splits, verbose=False)

In [60]:
items_df.head(3)

,ItemId,CreatorId,Year,DistributorId
0,121781,48864,2002,12008
1,106984,80902,2001,1409
2,2874,13251,1991,5225


In [61]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270151 entries, 0 to 270150
Data columns (total 4 columns):
ItemId           270151 non-null int64
CreatorId        270151 non-null int64
Year             270151 non-null object
DistributorId    270151 non-null int64
dtypes: int64(3), object(1)
memory usage: 8.2+ MB


##### Year

In [69]:
items_cross_val_scores(gl.SFrame(items_df.drop('Year',1)))

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6427693962686576

In [70]:
exp_items_df = items_df.copy()
exp_items_df.Year = exp_items_df.Year.replace('\D', 0, regex=True).astype(int)

items_cross_val_scores(gl.SFrame(exp_items_df))

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.637587166066135

In [75]:
exp_items_df = items_df.copy()
exp_items_df.Year = exp_items_df.Year.replace('\D', 0, regex=True).astype(int)
exp_items_df.Year.replace(0, int(round(exp_items_df[exp_items_df.Year > 0].Year.mean())), inplace=True) #mean

items_cross_val_scores(gl.SFrame(exp_items_df), 10)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

...

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6352462267197674

In [74]:
exp_items_df = items_df.copy()
exp_items_df.Year = exp_items_df.Year.replace('\D', 0, regex=True).astype(int)
exp_items_df.Year.replace(0, int(round(exp_items_df[exp_items_df.Year > 0].Year.median())), inplace=True) #median

items_cross_val_scores(gl.SFrame(exp_items_df), 10)

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

...

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6396509676634357

In [85]:
items_df.Year = items_df.Year.replace('\D', 0, regex=True).astype(int)
items_df.Year.replace(0, int(round(items_df[items_df.Year > 0].Year.mean())), inplace=True) #mean

##### CreatorId

In [95]:
len(items_df.CreatorId.unique()), len(users_df.UserId.unique())

(101589, 92106)

In [97]:
len(set(items_df.CreatorId.unique()) & set(users_df.UserId.unique()))

92106

In [106]:
items_cross_val_scores(gl.SFrame(items_df))

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6368300068803201

In [105]:
items_cross_val_scores(gl.SFrame(items_df.drop('CreatorId',1)))

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6450263456493919

##### DistributorId

In [99]:
len(items_df.DistributorId.unique()), len(users_df.UserId.unique())

(16731, 92106)

In [100]:
len(set(items_df.DistributorId.unique()) & set(users_df.UserId.unique()))

16731

In [107]:
items_cross_val_scores(gl.SFrame(items_df.drop('DistributorId',1)))

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = factorization_recommender

1.6458900536617669

##### Result

Profit:
   - replace 0 and non number values of _Year_ with mean value of postivive years
   
Bad:
   - _Year_ removal
   - _CreatorId_ removal
   - _DistributorId_ removal

## Crab

## Sklearn

# Model tuning

In [128]:
users_sf = gl.SFrame(users_df)

items_sf = gl.SFrame(items_df)

In [ ]:
cross_val_scores(lambda train_sf:\
                            gl.factorization_recommender.create(train_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                                                user_data=users_sf,
                                                                item_data=items_sf,
                                                                solver='sgd',
                                                                side_data_factorization=True,
                                                                regularization_type='weighted',
                                                                num_factors=12,
                                                                sgd_step_size=0.01,
                                                                max_iterations=110,
                                                                
                                                                linear_regularization=0.103448,
                                                                regularization=0.724138,
                                                                ), train_provided_sf, 10, verbose=False)

1.6166794833058433

In [ ]:
def evaluate_rmse(model, train_set, valid_set):
    return {'validation_rmse':graphlab.evaluation.rmse(valid_set['Rating'], model.predict(valid_set))}

cv_pairs = list(folds(train_provided_sf,3))

In [ ]:
job = gl.model_parameter_search.grid_search.create(cv_pairs, gl.factorization_recommender.create,\
                                                    {'user_id':'UserId','item_id':'ItemId','user_data':[users_sf],'item_data':[items_sf],'target':'Rating',
                                                     'solver':'sgd', 'regularization_type':'weighted',
                                                     'num_factors':list(range(1,16)),
                                                     'sgd_step_size':[0,.01,.001,.0001,.00001],
                                                     'regularization':list(np.linspace(1e-12,1,30)),
                                                     'linear_regularization':list(np.linspace(1e-12,1,30)),
                                                     'max_iterations':list(range(60,121,10))
                                                     }, evaluator=evaluate_rmse, return_model=False)

In [ ]:
job.get_status()

In [ ]:
search_result = job.get_results(wait=False)

In [ ]:
job.cancel()

## Checked models

p.s. sorted by score on kaggle

### Latent Factor (Graphlab)

Essential param:
- solver = 'sgd'
- side_data_factorization = True (default)
- regularization_type = 'weighted'

1.  1. Params:
       - solver = 'sgd
       - sgd_step_size = 0.01
       - max_iterations = 100
       - num_factors = 12
       - regularization = 0.724138
       - regularization_type = 'weighted'
       - side_data_factorization = True
    2. Results:
        - _Cross-val score: 1.6188531735562157_
        - **Kaggle score: 1.60929**

1.  1. Params - same as next but with side_data_factorization=True
    2. Results:
        - _Cross-val score: 1.6303911001573472_
        - **Kaggle score: 1.62233**
1. 
    1. Params:
        - solver='sgd'
        - sgd_step_size = 0
        - linear_reg = 0.758620689655
        - max_iterations = 70
        - nmf = 0
        - num_factors = 5
        - regularization = 0.413793103449
        - regularization_type = 'weighted'
        - side_data_factorization = False
    2. Results:
        - _Cross-val score: 1.6787204753271823_
        - **Kaggle score: 1.67161**

1. solver='sgd, sgd_step_size=.0001, side_data_factorization=False, regularization=9.9999999999999998e-13
    - _Cross-val score: 1.7613632832631356_, **Kaggle score: 1.75684**
1. solver='sgd, sgd_step_size=.001
    - _Cross-val score: 1.7869663402768563_, **Kaggle score: 1.76018**

# Recommendation

In [139]:
model = gl.factorization_recommender.create(train_provided_sf, user_id='UserId', item_id='ItemId', target='Rating', verbose=False,
                                            user_data=users_sf,
                                            item_data=items_sf,
                                            num_factors=12,
                                            sgd_step_size=0.01,
                                            max_iterations=100,
                                            solver='sgd',
                                            side_data_factorization=True,
                                            regularization_type='weighted',
                                            regularization=0.724138)

Recsys training: model = factorization_recommender

In [140]:
y_pred = model.predict(graphlab.SFrame(test_prefs_df)).to_numpy()

result = pd.DataFrame(data=hstack([test_prefs_df.PairId.as_matrix()[:,newaxis],y_pred[:,newaxis]]),columns=['PairId','Rating'])

result.PairId = result.PairId.astype(int)

In [141]:
result.describe()

,PairId,Rating
count,50000.000000,50000.000000
mean,24999.500000,7.634979
std,14433.901067,0.973897
min,0.000000,2.398545
25%,12499.750000,7.079103
50%,24999.500000,7.627352
75%,37499.250000,8.270776
max,49999.000000,11.222252


In [142]:
from datetime import datetime

result.to_csv('submission_%s.csv' % datetime.now().isoformat(),index=False)